## Import package

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from imageio import imread
from skimage.transform import resize
from scipy.spatial import distance
from keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

Using TensorFlow backend.


## Set directory and path

In [17]:
ROOT_DIR = os.getcwd() #main directory

IMG_DIR = os.path.join(ROOT_DIR, 'images') #images directory

print(IMG_DIR)

MODEL_DIR = os.path.join(ROOT_DIR, 'model') #model directory

CASCADE_PATH = os.path.join(MODEL_DIR, 'cv2', 'haarcascade_frontalface_alt2.xml') #cascade path

KERAS_PATH = os.path.join(MODEL_DIR ,'keras', 'facenet_keras.h5') #keras path

/home/lab513/face_128-dim/images


In [6]:
model = load_model(KERAS_PATH) #load keras model

In [7]:
name_list = os.listdir(IMG_DIR) #read the images people names
image_size = 160 #set image size

## Image whitening process, removes redundant information in the image, also highlights the features, and defines L2

In [10]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

## Image Processing

In [11]:
def load_and_align_images(filepaths, margin):
    cascade = cv2.CascadeClassifier(CASCADE_PATH)
    
    aligned_images = []
    for filepath in filepaths:
        img = imread(filepath)

        faces = cascade.detectMultiScale(img,
                                         scaleFactor=1.1,
                                         minNeighbors=3)
        (x, y, w, h) = faces[0]
        cropped = img[y-margin//2:y+h+margin//2,
                      x-margin//2:x+w+margin//2, :]
        aligned = resize(cropped, (image_size, image_size), mode='reflect')
        aligned_images.append(aligned)
            
    return np.array(aligned_images)

## Calculate embedding values

In [12]:
def calc_embs(filepaths, margin=10, batch_size=1):   ### !!!! 這裡要修改
    aligned_images = prewhiten(load_and_align_images(filepaths, margin))
    pd = []
    for start in range(0, len(aligned_images), batch_size):
        pd.append(model.predict_on_batch(aligned_images[start:start+batch_size]))
    embs = l2_normalize(np.concatenate(pd))

    return embs

## Test embedding values

In [19]:
calc_embs(['/home/lab513/face_128-dim/images/BillGates/Bill_Gates_0000.jpg'])  ###!!!

array([[-1.11268118e-01,  5.62553816e-02, -4.35333215e-02,
         8.02948475e-02, -5.32137183e-03,  3.16822305e-02,
        -1.55051723e-01, -8.39145854e-03,  1.23464011e-01,
        -1.57863021e-01, -9.87251842e-05,  2.95289960e-02,
        -1.21730231e-02,  1.33604124e-01,  4.42545824e-02,
         9.26484987e-02,  2.25569420e-02,  7.34831467e-02,
         3.35663441e-03,  1.88413829e-01,  1.17630959e-02,
        -3.61146801e-03,  5.50837889e-02,  8.04516971e-02,
        -1.36016896e-02,  1.52466837e-02, -1.02662660e-01,
         1.70878470e-02,  1.41111389e-01,  3.25863669e-03,
        -3.41998450e-02,  1.94938276e-02, -1.12565354e-01,
         1.29556522e-01, -6.93163946e-02,  4.67346273e-02,
         9.33347642e-03,  3.98831852e-02,  8.75391215e-02,
        -6.98434841e-03,  2.20776513e-01, -1.06723413e-01,
        -4.00678068e-02, -3.19848210e-02, -4.43502292e-02,
         3.93333286e-03,  5.86186908e-02, -2.44966611e-01,
         2.72554737e-02,  1.91387143e-02, -3.30376327e-0

## Define to calculate Euclidean distance

In [13]:
def calc_dist1(imgemb0,imgemb1):
    emb0 = [imgemb0]
    emb1 = [imgemb1]
    return distance.euclidean(calc_embs(emb0), calc_embs(emb1))

## Define the calculation and sort the pictures of each directory

In [20]:
def img_dist(image_dirpath): #output a list of img distances in img_path
    img_path = os.listdir(image_dirpath)    #讀出image_dirpath下的資料夾
    for i in range(len(img_path)):
        name_paths = os.path.join(image_dirpath,str(img_path[i]))  #用迴圈依序讀出各資料夾並拼接出絕對路徑
        sort_list = []                                             #/home/user/Downloads/keras-facenet/data/images/ + BillGates
        name_imgpaths = [os.path.join(name_paths, f) for f in os.listdir(name_paths)]#因為listdir只能讀路徑無法讀列表 所以必須把name_paths組合成絕對路徑
                                                                    #讀取目錄下的圖片再為他們拼接成絕對路徑#'/home/user/Downloads/keras-facenet/data/images/BillGates/ + Bill_Gates_0002 (copy).jpg'
        embs = calc_embs(name_imgpaths)
        n =len(name_imgpaths)
        print(n)
        for j in range(n-1):
            for k in range(n-j-1):
                #print(j,j+k+1)
                result = calc_dist1(name_imgpaths[j], name_imgpaths[j+k+1])
                sort_list.append(result)
        print(sorted(sort_list, key=float))
    return sorted(sort_list, key=float),embs

In [21]:
(ll, emb) = img_dist(IMG_DIR)

3
[0.5930255055427551, 0.618617832660675, 0.7355362176895142]
3
[1.1636340618133545, 1.3112479448318481, 1.4303568601608276]
3
[0.4924164414405823, 0.5091053247451782, 0.569210410118103]
3
[0.5079659819602966, 0.6593101620674133, 0.7331599593162537]
